# Simple factory
This makes better use of factory, because depending on arg passed to it, it creates a different type of step. Otherwise, we may as well us strategy pattern (only use of factory is to construct step later when configs etc are known - but a given factory always produces same kind of step, except from configuration).

In [ ]:
from typing import ClassVar


class StepFactoryInterface(ABC):
    @abstractmethod
    def create_step(self, step_name: str) -> ConfigurableRetryStep:
        ...

def load_step_config_dict_from_yaml(step_name: str): dict[str, Any]
    ...

class StepFactoryWrapper:
    def __init__(
        self,
        stepfactory_lookup_table: dict[str, StepFactoryInterface],
    ):
        self._stepfactory_lookup_table = stepfactory_lookup_table

    # def set_stepname_to_factory_map(
    #     self,
    #     stepname_to_factory_map: dict[str, StepFactoryInterface],
    # ):
    #     self._steptype_to_factory_map = stepname_to_factory_map

    # def update_stepname_to_factory_map(
    #     self,
    #     additional_stepname_to_factory_map: dict[str, StepFactoryInterface],
    # ):
    #     self._steptype_to_factory_map.update(
    #         additional_stepname_to_factory_map,
    #     )

    def create_step(self, step_name: str) -> ConfigurableRetryStep:
        # step name identifies config location.
        step_config: dict[str, Any] = load_step_config_dict_from_yaml(step_name=step_name)
        # Before validating config, we need to know for what type of step it is.
        # todo: Make knowable for typechecker that this key exists. Use typeddict (with optional keys)?
        step_type = step_config['step_type']
        # Look up which factory to use, based on step_type speified in config
        factory: StepFactoryInterface = self._stepfactory_lookup_table[step_type]
        return factory.create_step(step_config=step_config)


class _FrameworkProcessingStepFactory():
    def create_step(self, step_config: dict[str: Any]) -> ProcessingStep:
        ...

SyntaxError: invalid syntax (1812735520.py, line 39)

Note that the StepFactoryWrapper is decoupled from the specific StepFactory that will be used to create the step. The latter is determined by a lookup table, which is injected into to the StepFactoryWrapper during instantiation.

The downside is that this is less convenient for simple use cases, where the user is content with choosing only from the default factories that ship with the library. To remediate this disadvantage, we can simply create a facade, which instantiates the StepFactoryWrapper with the default lookup table. More advanced users, by contrast, can directly import this default lookup table and customize it to point to custom StepFactory implementations. In a second step, they then initialize the StepFactoryWrapper directly, passing it the custom lookup table.

In [ ]:
# higher-level-interface
# ======================

stepfactory_lookup_table: dict[str, StepFactoryInterface] = {
    'FrameworkProcessor': _FrameworkProcessingStepFactory,
}

# This is what user will import
stepfactory_wrapper = StepFactoryWrapper(
    stepfactory_lookup_table=stepfactory_lookup_table,
)

In [ ]:
# lower-level interface (if customization of factories is needed)
# ===============================================================

# Implement custom stepfactory
class  _CustomProcessingStepFactory():
    ...

# add it to the lookup table
stepfactory_lookup_table.update(
    {
        'CustomProcessor': _CustomProcessingStepFactory,
    },
)

# Instantiate StepFactory with customized lookup table
customized_step_factory = StepFactoryWrapper(
    stepfactory_lookup_table=stepfactory_lookup_table
)